In [45]:
from collections import defaultdict, deque
import random
import math
import time

import numpy as np
import pandas as pd

class City:
    id = 0
    def __init__(self):
        self.X = random.randint(-100, 100)
        self.Y = random.randint(-100, 100)
        self.Z = random.randint(0, 50)
        self.coordinates = np.array((self.X, self.Y, self.Z))
        self.id = City.id
        City.id += 1
    
class Graph:
    
    def __init__(self, cities):
        self.all_paths = []
        self.distance_list = []
        self.adjList = defaultdict(list)
        self.sym_matrix = self.generate_sym_matrix(cities)
        self.matrix_80 = self.generate_80_matrix(cities)
        self.asym100_matrix = self.generate_asym_matrix(cities)
        self.asym80_matrix = self.generate_80asym_matrix(cities)
        self.distance_matrix = np.zeros(self.sym_matrix.shape)

    #Stworzenie symetrycznej macierzy 100%

    def generate_sym_matrix(self, cities):
        sym_matrix = []
        for i in range(0, len(cities)):
            row = []
            for j in range(0, len(cities)):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            sym_matrix.append(row)
        return np.array(sym_matrix)
    

    #Stworzenie symetrycznej macierzy 80%

    def generate_80_matrix(self, cities):
        matrix = []
        for i in range(0, len(cities)):
            row = []
            for j in range(0, len(cities)):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            matrix.append(row)
        matrix = np.array(matrix)
        total_elements = matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(matrix.shape)
    

    #Stworzenie asymetrycznej macierzy 100%

    def generate_asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities)):
            row = []
            for j in range(0, len(cities)):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        return np.array(asym_matrix)
    

    #Stworzenie asymetrycznej macierzy 80%

    def generate_80asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities)):
            row = []
            for j in range(0, len(cities)):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        asym_matrix = np.array(asym_matrix)
        total_elements = asym_matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = asym_matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1 or matrix_flattened[index] == 2:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(asym_matrix.shape)


    #Metoda obliczajaca odleglosc bez uwzglednienia asymetrycznosci
    def calculate_distance(self, city1, city2):
        return np.linalg.norm(city1.coordinates - city2.coordinates)
    


    #Metoda obliczajaca odleglosc z uwzglednieniem asymetrycznosci
    def calculate_distance_asym(self, city1, city2):
        if city1.Z < city2.Z:
            #print("Going up")
            return 1.1 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        elif city1.Z > city2.Z:
            #print("Going down")
            return 0.9 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        else:
            #print("Same height")
            return np.linalg.norm(city1.coordinates - city2.coordinates)


    #Dodawanie krawedzi do grafu (do zmiennej adjList)
    def addEdge(self, u, v):
            self.adjList[u].append(v)
    
    #Dodawanie odleglosci (do zmiennej distance_matrix)
    def addDistance(self, i, j, cities, method):
        if method == 'sym':
            self.distance_matrix[i][j] = self.calculate_distance(cities[i], cities[j])
        elif method == 'asym':
            self.distance_matrix[i][j] = self.calculate_distance_asym(cities[i], cities[j])


    #Definiowanie krawedzi i przekazanie do metody addEdge oraz addDistance
    def createEdge(self, matrix, cities, method):
        for i in range(0, len(matrix)):
            for j in range(0, len(matrix)):
                if matrix[i][j] == 1 or matrix[i][j] == 2:
                    self.addEdge(i, j)
                    self.addDistance(i, j, cities, method)
                    #print(f"Created edge between {i} and {j}")


    def calculate_distance_from_path(self, path, cities, sym):
        total_distance = 0
        for i in range(0, len(path) - 1):
            if sym == 'sym':
                total_distance += self.calculate_distance(cities[path[i]], cities[path[i + 1]])
            elif sym == 'asym':
                total_distance += self.calculate_distance_asym(cities[path[i]], cities[path[i + 1]])
        return total_distance

    def ACO(self, cities, method):
        epochs = 100
        fx = len(cities)
        ants = 50 * fx
        f = np.array([0] * ants)
        ants_paths = []
        

        ants_paths = self.initial_scan(ants, ants_paths, fx, f)

        for _ in range(epochs):
            ants_paths, prob = self.epoch(ants_paths, f, fx, cities, method)

        prob = prob.index(max(prob))
        return ants_paths[prob], self.calculate_distance_from_path(ants_paths[prob], cities, method)
    
    def initial_scan(self, ants, ants_path, fx, f):
        currentNode = 0
        for _ in range(ants):
            # print(f'Ant {ant}')
            visited = set()
            visited.add(0)
            currentNode = 0
            path = [0] #* ants
            # print(ants_path)
            # print(path[i])
            i = 0
            
            # Warunek ktory sprawdza czy dlugosc nie jest wystarczajaca oraz czy nie przekroczono liczby iteracji
            while len(path) < fx and i < (3 * fx):
                # Warunek na dodanie koncowego powrotu do miasta 0 jezeli to mozliwe
                if len(path) == (fx - 1) and neighbour == 0:
                    path.append(neighbour)
                    break
                
                # Sprawdzenie czy aktualne miasto ma jakiegos sasiada oraz czy liczba sasiadow jest rowna dlugosci odwiedzonych
                if len(self.adjList[currentNode]) != 0 or len(self.adjList[currentNode]) != len(visited):
                    # print(f'Adj: {self.adjList[currentNode]}, Visted: {visited}')
                    # neighbour = np.random.choice(self.adjList[currentNode], p=[0.1, 0.1, 0.1, 0.7])

                    # Wybranie losowego sasiada
                    neighbour = np.random.choice(self.adjList[currentNode])


                    # print(f'Current node: {currentNode}, current neighoburs {self.adjList[currentNode]}, choice: {neighbour}')
                    # neighbour = self.adjList[currentNode][random_choice]
                    # print(f'Neighbour: {neighbour}')
                else: break
            
                # Dodawanie sasiada pod warunkiem, ze nie byl on juz odwiedzony
                if neighbour not in path:
                    path.append(neighbour)
                    # print(f'Neighbour node before: {neighbour})')
                    currentNode = neighbour
                    visited.add(neighbour)
                    # print(f'Current node after: {currentNode})')
                    # print(path)
                i += 1
            # print(f'Path: {path}')
            # Warunek sprawdzajacy czy dlugosc sciezki odpowiada ustalonej dlugosci
            if len(path) == fx:
                ants_path.append(path)
            # else:
                # print(f'\033[92mBad path: {path}\033[0m')
                

        return ants_path


    # Pojedyncza epoka
    def epoch(self, ants_paths, f, fx, cities, method):
        path_quantity = defaultdict(lambda: 0)
        probability = []
        sum_f = 0
        new_ants_paths = []
        new_probability = []

        for path in ants_paths:
            # Zliczane ile razy poszczegolna sciezka byla wybrana przez mrowki
            path_quantity[tuple(path)] += 1
            # print(path_quantity.keys())

        for key in path_quantity:
            # Obliczenie dlugosci sciezki
            dist = self.calculate_distance_from_path(key, cities, method)
            # Zapisanie ilosci odbytych sciezek przez mrowki
            quantity = path_quantity[key]
            # Dodanie do poszczegolnej sciezki odpowiednio: quanity, stosunek feromonu do dystansu pomnozony przez quanity
            path_quantity[key] = [quantity, dist, (fx/dist) * quantity * 100]
            # Sumowanie zostawionego feromonu [(fx/dist) * quantity * 100]
            sum_f += path_quantity[key][2]
        
        for i, path in enumerate(path_quantity):
            # Obliczenie szansy na wybranie danej sciezki ktora jest stosunkiem zostawionego feromonu do ich sumy
            probability.append(path_quantity[path][2]/sum_f)
            # print(f'Prob {i}: {probability[i]}')
        # print('\n')
        for _ in range(len(ants_paths)):
            # Wybranie sciezki bazujac jedynie na prawdopodobienstwu
            choice = np.random.choice(len(list(path_quantity.keys())), p=probability)
            # print(f'{choice}: {ants_paths[choice]}')
            # Dodawanie do list sciezkek oraz ich prawdopodobienstwa
            new_ants_paths.append(ants_paths[choice])
            new_probability.append(probability[choice])
        # print(f'\nNew prob: {new_probability}')
        return new_ants_paths, probability
    


    def ACO2(self, cities, method):
        epochs = 100
        fx = len(cities)
        ants = 64
        f = np.array([[0.0] * fx for _ in range(fx)])
        # print(f)
        all_paths, f = self.initial_scan2(cities, fx, f, ants)
        # print(f)
        # print(len(all_paths), f)
        path_number = len(all_paths)
        for _ in range(epochs):
            all_paths, f = self.epoch2(all_paths, f, fx, path_number)
            # print(f'\n\n{f}\n\n')

        shorthest_path, distance = self.find_shortest_path(f, fx, cities, method)


        return shorthest_path, distance
        


    def initial_scan2(self, cities, fx, f, ants):
        all_paths = [] 
        for _ in range(ants):
            path = np.array([0]*(fx + 1))
            visited = np.array([0]*(fx + 1))
            i = 1
            currentNode = 0
            current_dist = 0
            completed_path = False
            # print('\n')
            while i - 1 < fx:
                current_city = np.array(self.adjList[currentNode])
                acceptable_neigbours = current_city[~np.isin(current_city, visited)]

                # print(f'Current neigh: {current_city}, Visited: {visited}, Acceptable neigh: {acceptable_neigbours}')
                # print(f' i = {i}, fx = {fx}, {np.isin(0, current_city)}')

                if i  == fx and np.isin(0, current_city):
                    # print('Completed path')
                    completed_path = True

                    current_dist += self.distance_matrix[path[i - 1], 0]
                    # f[path[i - 1], 0] += (fx / current_dist) * 100

                    break

                if len(acceptable_neigbours) != 0:
                    choice = np.random.choice(acceptable_neigbours)
                    current_dist += self.distance_matrix[path[i - 1], choice]
                    # print(f'Current dist: {current_dist}, {(fx / current_dist) * 100}')

                    path[i] = choice
                    visited[i] = choice
                    currentNode = choice
                    # print(f'Current path: {path}')
                    # print(f'Current feromons: \n{f}')
                    i += 1
                    continue
                else: break
                
            if completed_path: 
                all_paths.append(path)
                sum_of_feromons = (fx / current_dist) * 100
                # print(sum_of_feromons, path, len(path))
                for city in range(len(path) - 1):
                    f[path[city]][path[city + 1]] += sum_of_feromons / len(path)
                # print(f'{f}\n')


        return all_paths, f


    def epoch2(self, all_paths, f, fx, path_number):
        new_paths = [] 

        for _ in range(path_number):
            path = np.array([0]*(fx + 1))
            visited = np.array([0]*(fx + 1))
            i = 1
            currentNode = 0
            completed_path = False
            # print('\n')
            while i - 1 < fx:
                current_city = np.array(self.adjList[currentNode])
                acceptable_neigbours = current_city[~np.isin(current_city, visited)]

                # print(f'Current neigh: {current_city}, Visited: {visited}, Acceptable neigh: {acceptable_neigbours}')
                # print(f' i = {i}, fx = {fx}, {np.isin(0, current_city)}')

                if i  == fx and np.isin(0, current_city):
                    # print('Completed path')
                    completed_path = True

                    current_dist = self.distance_matrix[path[i - 1], 0]

                    break

                if len(acceptable_neigbours) != 0:
                    # print(f'\n{f}\n')
                    prob = f[path[i - 1]][acceptable_neigbours]
                    # print(f'Prob row: {prob}')
                    sum_of_feromons = np.sum(prob)
                    # print(f'Prob: {prob}, {f[path[i - 1]][acceptable_neigbours]}')
                    if sum_of_feromons != 0:
                        for j in range(len(prob)):
                            # print(f'\nferomons: {prob[j]}, sum: {sum_of_feromons}\n' )
                            prob[j] = prob[j] / sum_of_feromons
                        # print(f'Prob: \n{prob}')
                    else: break

                    choice = np.random.choice(acceptable_neigbours, p=prob)
                    current_dist = self.distance_matrix[path[i - 1], choice]

                    # print(f'i: {i}')
                    path[i] = choice
                    visited[i] = choice
                    currentNode = choice
                    # print(f'Current path: {path}')
                    # print(f'Current feromons: \n{f}')
                    i += 1
                    continue
                else: break
                
            if completed_path: 
                new_paths.append(path)
                sum_of_feromons = (fx / current_dist) * 100
                # print(sum_of_feromons, path, len(path))
                for city in range(len(path) - 1):
                    f[path[city]][path[city + 1]] += sum_of_feromons / len(path)
                # print(f'{f}\n')
        
        f_10 = 0.1 * f
        f -= f_10
        return new_paths, f

    def find_shortest_path(self, f, fx, cities, method):
        path = np.array([0] * (fx + 1))
        visited = np.array([0] * (fx))
        # print(pd.DataFrame(f))
        currentNode = 0
        for i in range(len(f[0]) - 1):
            row = f[currentNode]
            row_adj = row[visited == 0]
            # print(f'New row: {row_adj},\n\n old: {row}, \n\nvisited: {visited}\n\n')
            max_row = np.max(row_adj[1:])
            index = np.where(row == max_row)
            city = index[0][0]
            # print(city)
            path[i + 1] = city
            visited[city] = city
            currentNode = city
        path[-1] = 0
        if method == 'sym':
            distance = self.calculate_distance_from_path(path, cities, 'sym')
        elif method == 'asym':
            distance = self.calculate_distance_from_path(path, cities, 'asym')

        # print(path)
        
        return path, distance

In [50]:
cities = []
for i in range(10):
    cities.append(City())
    # print(f'City {i}: X: {cities[i].X}, Y: {cities[i].Y}, Z: {cities[i].Z}')
graph = Graph(cities)

# print(graph.sym_matrix)
print(pd.DataFrame(graph.matrix_80))
# print(graph.asym100_matrix)
#print(graph.asym80_matrix)

# graph.createEdge(graph.sym_matrix, cities, 'sym')
graph.createEdge(graph.matrix_80, cities, 'sym')
# graph.createEdge(graph.asym100_matrix, cities, 'asym')
# graph.createEdge(graph.asym80_matrix, cities, 'asym')

print('\nACO')
start_time = time.perf_counter()
ACO_path, ACO_dist = graph.ACO(cities, 'sym')
print(f'Best path: {ACO_path}, distance: {ACO_dist}')
end_time = time.perf_counter()
print(f'ACO time: {(end_time - start_time)}')


print('\nACO 2')
start = time.perf_counter()
all_paths, distance = graph.ACO2(cities, 'asym')
print(all_paths, distance)
end_time = time.perf_counter()
print(f'ACO2 time: {(end_time - start_time)}')

   0  1  2  3  4  5  6  7  8  9
0  0  1  1  1  1  1  1  1  1  0
1  1  0  1  0  1  1  1  1  1  1
2  1  0  0  1  1  1  1  1  1  1
3  0  1  1  0  1  1  1  0  1  0
4  1  1  1  1  0  1  1  1  1  1
5  1  1  1  1  0  0  0  0  0  1
6  1  0  0  1  1  1  0  1  1  1
7  1  1  1  1  1  1  1  0  1  0
8  1  1  0  0  1  0  1  0  0  1
9  1  1  1  1  1  1  1  1  1  0

ACO
Best path: [0, 6, 3, 4, 8, 9, 1, 5, 2, 0], distance: 1016.9157676718692
ACO time: 0.3712509000033606

ACO 2
[0 5 1 6 7 2 3 8 9 4 0] 1016.3681809624222
ACO2 time: 1.6991950999945402
